In [1]:
import os

In [2]:
%pwd

'd:\\end_to_end_ml_project\\reasearch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\end_to_end_ml_project'

In [5]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen= True)

class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from Cnn_Classifier.constant import *
from Cnn_Classifier.utils.common import  read_yaml , create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
          config = self.config.prepare_callbacks
          model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
          create_directories([
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)
          ])

          prepare_callback_config = PrepareCallbacksConfig(
                root_dir= Path(config.root_dir),
                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath= Path(config.checkpoint_model_filepath)

          )

          return prepare_callback_config

In [8]:
import os
import urllib.request as request
from zipfile import  ZipFile
import tensorflow as tf
import time

In [18]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

   
    @property
    def _create_ckpt_callbacks(self):
    # Convert the Path object to a string
        checkpoint_path_str = str(self.config.checkpoint_model_filepath)

        return tf.keras.callbacks.ModelCheckpoint(
           filepath=checkpoint_path_str,
           save_best_only=True,
    )


    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


In [19]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    prepare_callbacks_config = config.get_prepare_callback_config()
    


    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    

except Exception as e:
    raise e                                  

[2023-11-09 22:29:26,104: INFO: common]: yaml file:config\config.yaml loaded successfully
[2023-11-09 22:29:26,111: INFO: common]: yaml file:params.yaml loaded successfully
[2023-11-09 22:29:26,114: INFO: common]: directory:artifacts created successfully
[2023-11-09 22:29:26,117: INFO: common]: directory:artifacts\prepare_callbacks\checkpoint_dir created successfully
[2023-11-09 22:29:26,120: INFO: common]: directory:artifacts\prepare_callbacks\tensorboard_log_dir created successfully


In [20]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-11-09-22-32-32'